In [2]:
from itertools import product

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go

# Util

In [3]:
spectral_conditional_name = "cond_s"
volume_conditional_name = "cond_v"
angle_conditional_name = "cond_a"
matrix_size_name = "matrix_size"
offset_name = "offset"

matrix_offset_error_name = "matrix_offset_error"
right_part_offset_error_name = "right_part_offset_error"
both_parts_offset_error_name = "both_parts_offset_error"

In [3]:
def calculate_spectral_condition_number(matrix):
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix):
    volume_denominator = abs(linalg.det(matrix))

    volume_numerator = 1
    for row in matrix:
        volume_numerator *= linalg.norm(row)

    return volume_numerator / volume_denominator


def calculate_angle_condition_number(matrix):
    candidates = []
    for row, column in zip(matrix, linalg.inv(matrix).T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)

In [5]:
def run_benchmark(matrix, right_part, expected_solution, offset):
    spectral_condition_number = calculate_spectral_condition_number(matrix)
    volume_condition_number = calculate_volume_condition_number(matrix)
    angle_condition_number = calculate_angle_condition_number(matrix)

    changed_matrix = matrix - offset
    changed_right_part = right_part - offset

    changed_matrix_solution = linalg.solve(changed_matrix, right_part)
    changed_right_part_solution = linalg.solve(matrix, changed_right_part)
    changed_linear_system_solution = linalg.solve(changed_matrix, changed_right_part)

    return pd.Series(
        [
            spectral_condition_number,
            volume_condition_number,
            angle_condition_number,
            linalg.norm(changed_matrix_solution - expected_solution),
            linalg.norm(changed_right_part_solution - expected_solution),
            linalg.norm(changed_linear_system_solution - expected_solution),
        ],
    )

In [6]:
def draw_compare_plot_for_size_and_cond(data) -> None:
    fig = go.Figure()

    data = data[[matrix_size_name, spectral_conditional_name, volume_conditional_name, angle_conditional_name]].drop_duplicates()

    fig.add_scatter(
        x=data[matrix_size_name], 
        y=data[spectral_conditional_name],
        name="Спектральный критерий",
    )

    fig.add_scatter(
        x=data[matrix_size_name], 
        y=data[volume_conditional_name],
        name="Объёмный критерий",
    )

    fig.add_scatter(
        x=data[matrix_size_name], 
        y=data[angle_conditional_name],
        name="Угловой критерий",
    )

    fig.update_xaxes(title="Размер матрицы")
    fig.update_yaxes(title="Число обусловленности", tickformat='.2e')
    fig.update_layout(title="Зависимость числа обусловленности от размера матрицы")

    fig.show()

In [7]:
def draw_compare_plot_for_error_and_cond(data, selected_offset, error_name=both_parts_offset_error_name):
    fig = go.Figure()

    selected_data = data[data[offset_name] == selected_offset]
    fig.add_scatter(
        x=selected_data[spectral_conditional_name],
        y=selected_data[error_name],
        name='Спектральный критерий',
    )

    fig.add_scatter(
        x=selected_data[volume_conditional_name],
        y=selected_data[error_name],
        name='Объёмный критерий',
    )

    fig.add_scatter(
        x=selected_data[angle_conditional_name],
        y=selected_data[error_name],
        name='Угловой критерий',
    )

    fig.update_xaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title=f'Зависимость ошибки от числа обусловленности ({selected_offset = :.0e}, {error_name = })')

    fig.show()

In [8]:
def generate_matrix(element_factory, size):
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ]
    )

# Диагональная матрица

In [10]:
def get_spectral_bad_linear_system(size):
    matrix = np.eye(size)

    for i in range(size):
        matrix[i][i] = i + 1

    expected_solution = np.ones(size)
    right_part = np.dot(matrix, expected_solution)

    return matrix, right_part, expected_solution

In [11]:
spectral_bad_data = pd.DataFrame(product(range(3, 11), [10 ** -i for i in range(2, 11)]), columns=[matrix_size_name, offset_name])
spectral_bad_data[
    [
        spectral_conditional_name, 
        volume_conditional_name, 
        angle_conditional_name, 
        matrix_offset_error_name,
        right_part_offset_error_name,
        both_parts_offset_error_name
    ]
] = spectral_bad_data.apply(
    func=lambda row: run_benchmark(*get_spectral_bad_linear_system(int(row[matrix_size_name])), row[offset_name]),
    axis=1,
)
spectral_bad_data.sort_values(by=both_parts_offset_error_name, ascending=False)

,matrix_size,offset,cond_s,cond_v,cond_a,matrix_offset_error,right_part_offset_error,both_parts_offset_error
63,10,1.000000e-02,10.0,1.0,1.0,1.282460e-01,1.244897e-02,1.154214e-01
54,9,1.000000e-02,9.0,1.0,1.0,1.149300e-01,1.240874e-02,1.021600e-01
45,8,1.000000e-02,8.0,1.0,1.0,1.016334e-01,1.235889e-02,8.892921e-02
36,7,1.000000e-02,7.0,1.0,1.0,8.835965e-02,1.229552e-02,7.573684e-02
27,6,1.000000e-02,6.0,1.0,1.0,7.511375e-02,1.221224e-02,6.259479e-02
...,...,...,...,...,...,...,...,...
44,7,1.000000e-10,7.0,1.0,1.0,8.606862e-10,1.229552e-10,7.377310e-10
35,6,1.000000e-10,6.0,1.0,1.0,7.327347e-10,1.221224e-10,6.106122e-10
26,5,1.000000e-10,5.0,1.0,1.0,6.048990e-10,1.209798e-10,4.839192e-10
17,4,1.000000e-10,4.0,1.0,1.0,4.772607e-10,1.193152e-10,3.579456e-10


In [12]:
draw_compare_plot_for_size_and_cond(spectral_bad_data)

In [13]:
draw_compare_plot_for_error_and_cond(spectral_bad_data, selected_offset=1e-2)
draw_compare_plot_for_error_and_cond(spectral_bad_data, selected_offset=1e-4)
draw_compare_plot_for_error_and_cond(spectral_bad_data, selected_offset=1e-8)

# Трехдиагональная матрица

In [15]:
def get_volume_bad_linear_system(size):
    matrix = np.eye(size) * 2 + np.eye(size, k=-1) * -1 + np.eye(size, k=1) * -1
    
    expected_solution = np.ones(size)
    right_part = np.dot(matrix, np.ones(size))

    return matrix, right_part, expected_solution

In [16]:
volume_bad_data = pd.DataFrame(product(range(3, 11), [10 ** -i for i in range(2, 11)]), columns=[matrix_size_name, offset_name])
volume_bad_data[[spectral_conditional_name, volume_conditional_name, angle_conditional_name, matrix_offset_error_name, right_part_offset_error_name, both_parts_offset_error_name]] = volume_bad_data.apply(
    func=lambda row: run_benchmark(*get_volume_bad_linear_system(int(row[matrix_size_name])), row[offset_name]),
    axis=1,
)
volume_bad_data.sort_values(by=both_parts_offset_error_name, ascending=False)

,matrix_size,offset,cond_s,cond_v,cond_a,matrix_offset_error,right_part_offset_error,both_parts_offset_error
63,10,1.000000e-02,48.374150,589.090909,12.898203,3.663332e+01,3.663332e-01,3.296999e+01
54,9,1.000000e-02,39.863458,264.544892,11.291590,1.484541e+01,2.886607e-01,1.319592e+01
45,8,1.000000e-02,32.163437,120.000000,9.545214,4.436215e+00,2.218107e-01,3.881688e+00
36,7,1.000000e-02,25.274142,55.113519,8.124038,1.994120e+00,1.652271e-01,1.709246e+00
27,6,1.000000e-02,19.195669,25.714286,6.546537,9.860133e-01,1.183216e-01,8.216777e-01
...,...,...,...,...,...,...,...,...
35,6,1.000000e-10,19.195669,25.714286,6.546537,7.099296e-09,1.183216e-09,5.916080e-09
7,3,1.000000e-09,5.828427,3.061862,3.000000,8.746428e-09,2.915476e-09,5.830952e-09
26,5,1.000000e-10,13.928203,12.247449,5.338539,4.023369e-09,8.046740e-10,3.218695e-09
17,4,1.000000e-10,9.472136,6.000000,3.949684,2.039608e-09,5.099021e-10,1.529706e-09


In [17]:
draw_compare_plot_for_size_and_cond(volume_bad_data)

In [18]:
draw_compare_plot_for_error_and_cond(volume_bad_data, selected_offset=1e-2)
draw_compare_plot_for_error_and_cond(volume_bad_data, selected_offset=1e-4)
draw_compare_plot_for_error_and_cond(volume_bad_data, selected_offset=1e-8)

# Матрицы Гильберта

In [21]:
def get_hilbert_linear_system(size):
    matrix = np.around(generate_matrix(lambda row, column: 1 / (row + column - 1), size), 10)
    
    expected_solution = np.ones(size)
    right_part = np.dot(matrix, expected_solution)

    return matrix, right_part, expected_solution

In [22]:
hilbert_data = pd.DataFrame(product(range(2, 11), [10 ** -i for i in range(2, 11)]), columns=[matrix_size_name, offset_name])
hilbert_data[
    [
        spectral_conditional_name, 
        volume_conditional_name, 
        angle_conditional_name, 
        matrix_offset_error_name, 
        right_part_offset_error_name, 
        both_parts_offset_error_name
    ]
] = hilbert_data.apply(
    func=lambda row: run_benchmark(*get_hilbert_linear_system(int(row[matrix_size_name])), row[offset_name]),
    axis=1,
)
hilbert_data.sort_values(by=both_parts_offset_error_name, ascending=False)

,matrix_size,offset,cond_s,cond_v,cond_a,matrix_offset_error,right_part_offset_error,both_parts_offset_error
72,10,1.000000e-02,1.029944e+11,2.224547e+46,1.250090e+10,2.357692e+05,1.160266e+04,2.121923e+05
54,8,1.000000e-02,1.409990e+10,6.984559e+29,1.871414e+09,6.224481e+04,2.881633e+03,5.446421e+04
63,9,1.000000e-02,5.253769e+10,6.387268e+37,5.949241e+09,4.821317e+04,1.664714e+03,4.285615e+04
73,10,1.000000e-03,1.029944e+11,2.224547e+46,1.250090e+10,1.222345e+04,1.160266e+03,1.100111e+04
45,7,1.000000e-02,4.753813e+08,1.420976e+22,7.162373e+07,7.196395e+03,5.242900e+02,6.168338e+03
...,...,...,...,...,...,...,...,...
26,4,1.000000e-10,1.551374e+04,9.370861e+05,4.020914e+03,9.433226e-08,2.358304e-08,7.074921e-08
6,2,1.000000e-08,1.928147e+01,8.062258e+00,8.062258e+00,1.264911e-07,6.324555e-08,6.324555e-08
17,3,1.000000e-10,5.240568e+02,7.580470e+02,1.728872e+02,1.156070e-08,3.853584e-09,7.707126e-09
7,2,1.000000e-09,1.928147e+01,8.062258e+00,8.062258e+00,1.264911e-08,6.324555e-09,6.324555e-09


In [23]:
draw_compare_plot_for_size_and_cond(hilbert_data)

In [24]:
draw_compare_plot_for_error_and_cond(hilbert_data, selected_offset=1e-2)
draw_compare_plot_for_error_and_cond(hilbert_data, selected_offset=1e-4)
draw_compare_plot_for_error_and_cond(hilbert_data, selected_offset=1e-8)